In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
from glob import glob
from tqdm import tqdm
from IPython.display import display

In [3]:
glob('AIHub/*.xlsx')

['AIHub/3_문어체_뉴스(3)_191213.xlsx',
 'AIHub/1_구어체(1)_191213.xlsx',
 'AIHub/3_문어체_뉴스(2)_191213.xlsx',
 'AIHub/3_문어체_뉴스(1)_191213.xlsx',
 'AIHub/4_문어체_한국문화_191213.xlsx',
 'AIHub/2_대화체_191213.xlsx',
 'AIHub/5_문어체_조례_191213.xlsx',
 'AIHub/3_문어체_뉴스(4)_191213.xlsx',
 'AIHub/6_문어체_지자체웹사이트_191213.xlsx',
 'AIHub/1_구어체(2)_191213.xlsx']

In [4]:
colum_mapping = {
    'ID': 'id',
    'SID': 'id',
    '제목': 'title', 
    'URL': 'url', 
    '원문':'korean', 
    'ko':'korean', 
    'NMT': 'nmt',
    'NMT 번역': 'nmt',
    'PE': 'english',
    'en': 'english',
    '분야': 'domain',
    '언론사': 'media',
    '한국어': 'korean',
    '영어': 'english',
    '날짜': 'date',
    '자동분류1': 'auto_label_1',
    '자동분류2': 'auto_label_2',
    '자동분류3': 'auto_label_3',
    '대분류': 'category_1',
    '소분류': 'category_2',
    '상황': 'situation',
    'Set Nr.': 'dialog_id',
    '발화자': 'speaker',
    '영어 검수': 'inspection',
    '번역문': 'english',
    '키워드': 'keyword',
    '지자체': 'location',
}

In [5]:
from uuid import uuid4

pbar = tqdm(glob('AIHub/*.xlsx'))

total = 0

df_list = {}
for f in pbar:
    sheet_names = pd.ExcelFile(f).sheet_names
    
    for name in sheet_names:
        pbar.set_description_str('{} / {}'.format(f, name))
        
        f_name = f.split('/')[-1].split('_')[1].strip()
        f_name = re.sub(r'\(\d+\)', '', f_name)
                 
        df = pd.read_excel(f, sheet_name=name).fillna('').rename(columns=colum_mapping)
        
        token = re.sub(r'\(\d+\)', '', f.split('/')[-1].split('.')[0]).split('_')
        
        df.at[:, 'source'] = 'aihub'
        
        df.at[:, 'style'] = token[1]
        if token[2].isdecimal() is False:
            if token[2] == '뉴스':
                df.at[:, 'theme'] = 'news'
            else:
                df.at[:, 'theme'] = token[2]
        
        display(df.head())
        
        total += len(df)

        df.to_json(
            'AIHub/{}.{}.json.bz2'.format(f_name, str(uuid4())[:5]),
            force_ascii=False,
            compression='bz2',
            orient='records',
            lines=True,
        )

print({'total', '{:,}'.format(total)})        

AIHub/3_문어체_뉴스(3)_191213.xlsx / Sheet1:   0%|          | 0/10 [00:14<?, ?it/s]

,id,date,auto_label_1,auto_label_2,auto_label_3,url,media,korean,english,source,style,theme
0,141833,20180705,"사회,사회일반",,,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,국제구호개발 NGO인 ㈔사랑광주리(이사장 오정현 목사)가 최근 수해가 발생한 지역을...,(Corporation) Love Box (Chief Pastor Oh Jung-h...,aihub,문어체,news
1,1203595,20190316,"경제,경제일반",,,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,‘찾아가는 교육 프로그램’은 한국에서 학교생활의 적응을 위해 노력하는 다문화 예비학...,Educational Program' provides students with em...,aihub,문어체,news
2,2446514,20181014,"국제,중국","국제,미국_북미",,http://www.hani.co.kr/arti/international/china...,한겨레,중국인 오페어를 고용한 작가 미아 리버튼 앨퍼트는 “(중국이) 분명히 글로벌 초강대...,"""(China) is clearly a global superpower, so it...",aihub,문어체,news
3,98515,20180625,"사회,사건_사고","사회,의료_건강",,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,목격자는 “(최씨가) 계좌이체를 하겠다고 했는데 (주유소) 사장님이 통장에 안 들어...,"""(Choi) Said he would transfer through his acc...",aihub,문어체,news
4,1662830,20180831,"사회,교육_시험","정치,정치일반",,http://www.naeil.com/news_view/?id_art=287186,내일신문,유 후보자는 “(교육정책은) 속도가 아니라 방향이 중요하다”며 “안정된 교육개혁을 ...,"""(Education policy) is important not speed, bu...",aihub,문어체,news


AIHub/1_구어체(1)_191213.xlsx / Sheet1:  10%|█         | 1/10 [00:50<06:33, 43.73s/it]   

,id,korean,english,source,style
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...,aihub,구어체
1,2,씨티은행에서 일하세요?,Do you work at a City bank?,aihub,구어체
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ...",aihub,구어체
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...,aihub,구어체
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...,aihub,구어체


AIHub/3_문어체_뉴스(2)_191213.xlsx / 뉴스(100,127):  20%|██        | 2/10 [01:15<04:47, 35.91s/it]

...

,id,location,korean,english,source,style,theme
0,261811,경기도,"""경기도가 말산업 육성을 위해 총예산 245,193천원으로 2013년 경기도 용인시...","""The Gyeonggi provincial government announced ...",aihub,문어체,지자체웹사이트
1,409852,경기도,"""경기도가 주최하고 경기FTA활용지원센터와 코트라가 주관한 이번 시장개척단은 지난 ...","""Organized by Gyeonggi provincial government a...",aihub,문어체,지자체웹사이트
2,352671,경기도,"""경기도가 주최하고 경기도비정규직지원센터가 주관한 이번 교육은 공공 부문이 직·간접...","""Organized by Gyeonggi provincial government a...",aihub,문어체,지자체웹사이트
3,308191,경기도,"""경기도가 주최하고 사단법인 한국장애인복지시설협회 경기도협회(대표자:정권)에서 주관...","""Organized by Gyeonggi provincial government a...",aihub,문어체,지자체웹사이트
4,185424,경기도,"""경기도가 주최하고 인구보건복지협회 경기지회, 아이낳기좋은세상 경기운동본부가 주관하...","""Hosted by Gyeonggi provincial government, org...",aihub,문어체,지자체웹사이트


AIHub/1_구어체(2)_191213.xlsx / Sheet1:  90%|█████████ | 9/10 [04:06<00:23, 23.35s/it]     

,id,korean,english,source,style
0,200001,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t...",aihub,구어체
1,200002,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...,aihub,구어체
2,200003,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version.",aihub,구어체
3,200004,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ...",aihub,구어체
4,200005,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra...",aihub,구어체


AIHub/1_구어체(2)_191213.xlsx / Sheet1: 100%|██████████| 10/10 [04:17<00:00, 25.71s/it]

{'1,602,708', 'total'}


In [ ]:
from glob import glob
from bitext_exports import BitextExports

In [ ]:
index = 'corpus-bitext-aihub'

utils = BitextExports()

for f in glob('AIHub/*.json.bz2'):
    if f.find('morp') > 0:
        continue

    utils.import_bitext(filename=f, index=index)